# Zad. 

Otagować (czyli wskazać słowa "kluczowe") teksty korpusu 20newsgroups

In [ ]:
from gensim import corpora, models, similarities, matutils
import numpy as np
import os 

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

Wypisz pierwszy tekst

In [ ]:
newsgroups_train.data[0]

In [ ]:
print(newsgroups_train.data[0])

In [ ]:
print(len(newsgroups_train.data))

Wypisz tagi (numery) i ich nazwy (napisy) 

In [ ]:
print(newsgroups_train.target)
print(len(newsgroups_train.target))
print(len( set( newsgroups_train.target ) ))

In [ ]:
print(len(newsgroups_train.target_names))
newsgroups_train.target_names

### Można sczytać dane bez nagłówków i stopek

In [ ]:
to_remove= ('headers', 'footers', 'quotes')

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='all', shuffle=True, remove = to_remove)

In [ ]:
newsgroups_train.data[0]

In [ ]:
print(newsgroups_train.data[0])

In [ ]:
newsgroups_train.target_names

In [ ]:
print(newsgroups_train.target)

# Zad
Wykonaj przekształcenie Tf-Idf

In [ ]:
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

def remove_punctuation(words):
    remove_list = list(string.punctuation) + ["--", "\"", "``" , "...", "''", "'" ]
    return [w for w in words if not w in set(remove_list)]

def remove_nummbers(words):
    return [w for w in words if not w.isdigit()]

def to_lower(words):
    return [w.lower() for w in words]


def my_tokenizer(sent):
    tokens = word_tokenize(sent)
    tokens = remove_punctuation(tokens)
    tokens = remove_nummbers(tokens)
    tokens = to_lower(tokens)
    stemmer = PorterStemmer()
    res = [stemmer.stem(word) for word in tokens]
    return res 


tfidf_vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, stop_words=stopwords.words('english') + list(string.punctuation))
tfidf_matrix = tfidf_vectorizer.fit_transform(newsgroups_train.data)
# print( tfidf_vectorizer.vocabulary_ )

In [ ]:
print("----")
print(tfidf_matrix.todense().shape)
print(tfidf_matrix.todense())

In [ ]:
corpus_sparse = matutils.Sparse2Corpus(tfidf_matrix, documents_columns=False)
dictionary_list = tfidf_vectorizer.get_feature_names()
dictionary_sparse_dict = {i:j for i,j in enumerate(dictionary_list)}
dictionary = corpora.Dictionary.from_corpus(corpus_sparse, id2word=dictionary_sparse_dict)

In [ ]:
model = models.LdaModel(corpus=corpus_sparse, id2word=dictionary, num_topics=30)#, alpha='auto', eta='auto', iterations=100)

In [ ]:
print(len(model.get_topics()))
model.get_topics()

In [ ]:
import numpy as np

topic_inex = 1
topics = np.argsort(model.get_topics()[topic_inex,:])[::-1] #::-1 sortowanie w odwrotnej kolejności
for x in topics[:5]:
    print(dictionary[x])

In [ ]:
model.print_topic(topicno=topic_inex)

In [ ]:
topic2_terms = model.get_topic_terms(topicid=topic_inex)
topic2_words = [
    (dictionary.get(i), j)
    for i,j in topic2_terms
]
topic2_words

# Wizualizacja modelu LDA:

pyLDAvis

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(model, corpus_sparse, dictionary)